
# 🧮 Notebook 04 – Mapping Symbolic Metrics from SWOW Graph

This notebook computes symbolic cognitive metrics from the SWOW semantic network graph, including:

- **Anchoring Alpha (α)**: Semantic centrality  
- **Curvature Kappa (κ)**: Local symbolic clustering  
- **Recursive Entropy (Eᵣ)**: Symbolic redundancy via node degree  

The resulting symbolic metrics are exported as a `.csv` file for subsequent manifold construction and UMAP visualization in notebooks 05 and 06.

**Output**: `symbolic_metrics_embeddings.csv`


# 🧠 Notebook 04 – Mapping Symbolic Metrics from SWOW

This notebook extracts empirical symbolic metrics from the SWOW graph, computes node-level statistics (degree, centrality, clustering), and builds a DataFrame aligned with the symbolic manifold axes: anchoring, curvature, and entropy.

In [ ]:
import pandas as pd
import networkx as nx
from pathlib import Path

# Load the SWOW graph
path = Path("/Users/demetriosagourakis/Library/Mobile Documents/com~apple~CloudDocs/Biologia Fractal/entropic-symbolic-society/NHB_Symbolic_Mainfold/data/swow_graph.gpickle")
G = nx.read_gpickle(path)
print(f"Graph loaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

### 🔢 Compute centrality and clustering metrics
These are used as proxies for symbolic manifold axes.

In [ ]:
# Anchoring: closeness centrality
# Curvature: clustering coefficient (local divergence)
# Entropy: degree variability (connectivity diversity)

centrality = nx.closeness_centrality(G)
clustering = nx.clustering(G)
degree = dict(G.degree())

### 📊 Create symbolic metrics DataFrame
Each row corresponds to a word/node with its symbolic coordinates.

In [ ]:
import pandas as pd
df_metrics = pd.DataFrame({
    "word": list(centrality.keys()),
    "anchoring_alpha": [centrality[n] for n in centrality],
    "curvature_kappa": [clustering[n] for n in clustering],
    "entropy_Er": [degree[n] for n in degree],
})

df_metrics.sort_values("anchoring_alpha", ascending=False).head()
#df_metrics.to_csv("/Users/demetriosagourakis/Library/Mobile Documents/com~apple~CloudDocs/Biologia Fractal/entropic-symbolic-society/NHB_Symbolic_Mainfold/data/symbolic_metrics_embeddings.csv", index=False)

# ✅ 1. UMAP com 10 dimensões sobre as três métricas principais
from umap import UMAP

umap_model = UMAP(n_components=10, metric='euclidean', n_neighbors=15, min_dist=0.1, random_state=42)
X_umap = umap_model.fit_transform(df[['anchoring_alpha', 'curvature_kappa', 'entropy_Er']])

# ✅ 2. Adiciona colunas emb_0 a emb_9
for i in range(X_umap.shape[1]):
    df[f'emb_{i}'] = X_umap[:, i]

# ✅ 3. Salva CSV com todas as colunas para uso posterior
output_path = DATA / 'symbolic_metrics_embeddings.csv'
df.to_csv(output_path, index=False)

print(f'Saved: {output_path} with shape {df.shape}')